# Demo: Serenity Derivatives API - Yield Curve and Vol Surface

Serenity builds in sophisticated option and rates analytics as part of its core offering, and these functions
are all exposed via the API. This notebook shows how you can use it to fetch yield curves and volatility
surfaces.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
from os import getenv
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(getenv('SERENITY_CONFIG_ID', None))

In [ ]:
from datetime import datetime, timedelta
from uuid import UUID, uuid4
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import numpy as np
import pandas as pd

from serenity_sdk.renderers.derivatives.widget_tools import YieldCurveVersionTimeChooser, VolatilitySurfaceVersionTimeChooser
from serenity_sdk.renderers.derivatives.table_plot import YieldCurveTablePlot, VolatilitySurfaceTablePlot
from serenity_types.pricing.derivatives.rates.yield_curve import YieldCurveVersion

# plot parameters
plt.rcParams['font.size'] = '16'

# create an alias to the api
api = connect_widget.get_api()

In [ ]:
# Pick a datetime.
# WIP: need to give some buffer on the end_datetime at the time of writing this notebook
import pytz
end_dt =datetime.utcnow() - timedelta(hours=3)
start_dt = end_dt - timedelta(days=2)

# Yield Curves

## Pick Version
There are multiple type of market data built on multiple datetimes (e.g. hourly, daily). 
Pick the version to show and plot. 

In [ ]:
versions = api.pricer().get_available_yield_curve_versions()

In [ ]:
versions = api.pricer().get_available_yield_curve_versions(
    start_datetime=start_dt, end_datetime=end_dt)
vtc_yc = YieldCurveVersionTimeChooser(versions)

print('Pick the curve and as-of-time from the drop-down menus')
display(vtc_yc.get_widget_to_display())

## Now, load the selected version

In [ ]:
from serenity_sdk.renderers.derivatives.converters import convert_object_list_to_df

In [ ]:
# load the version through API
id_selected, as_of_time_selected = vtc_yc.get_id_as_of_time()
yc_selected = api.pricer().get_yield_curve_version(id_selected, as_of_time_selected)
print(f'Loaded id={id_selected}, as-of-time={as_of_time_selected}')

# use a helper object to show the content
yc_tp = YieldCurveTablePlot(yc_selected)

### Peeking data

In [ ]:
# peeking raw data
yc_tp.raw_pts.head(2)

In [ ]:
# peeking interpolated data
yc_tp.interpolated_curve.head(2)

## Plotting
* Top: future prices and corresponding index (underlying) spot price. 
* Middle: The flat-forward interpolation method is used to get the interpolated points.
* Bottom: It is equivalent to the log-linear interpolation in the discount factor.

In [ ]:
print('There are small variations in mark price spot across expiries - probably due to timing of the market data collection')
yc_tp.plot()
plt.show()

# Volatility Surfaces

## Pick Version
There are multiple type of market data built on multiple datetimes (e.g. hourly, daily). 
Pick the version to show and plot. 

In [ ]:
# Load available vol surfaces
versions = api.pricer().get_available_volatility_surface_versions(
    start_datetime=start_dt, end_datetime=end_dt)
vtc_vol =  VolatilitySurfaceVersionTimeChooser(versions)

print('Pick the vol surface and build time from the drop-down menus')
vtc_vol.widget_name.value = vtc_vol.widget_name.options[1] # choose the log_money version
display(vtc_vol.get_widget_to_display())

## Now, load the selected version

In [ ]:
# load the version through API
id_selected, as_of_time_selected = vtc_vol.get_id_as_of_time()
vs_selected = api.pricer().get_volatility_surface_version(id_selected, as_of_time_selected)
print(f'Loaded id={id_selected}, as-of-time={as_of_time_selected}')


# use a helper object to show the content 
vs_tp = VolatilitySurfaceTablePlot(vs_selected)

### Peeking data

In [ ]:
# showing available expiries
vs_tp.time_to_expiries

In [ ]:
# peeking raw data
vs_tp.raw_pts.head(2)

In [ ]:
# peeking interpolated data
vs_tp.interpolated_surface.head(2)

In [ ]:
# show calibration SVI parameters
df = pd.DataFrame(vs_tp.vs.interpolated.calibration_params).T
df.index.name = 'time-to-expiry'
df

## Plotting
For each expiries, show a smile curve with input data (dots)

In [ ]:
vs_tp.plot()

# 3D interactive plot

In [ ]:
# interpolated surface
ivs = vs_selected.interpolated
interpolated_df = pd.DataFrame(
    {
        'expiry':ivs.time_to_expiries,
        'strike':ivs.strikes,
        'vol':ivs.vols
    })
interpolated_df = interpolated_df.groupby(['expiry', 'strike'])['vol'].mean().unstack()
T, K, sigma = interpolated_df.index.values, interpolated_df.columns.values, interpolated_df.values.T

raw_df = convert_object_list_to_df(vs_selected.raw.vol_points)
T_raw, K_raw, sigma_raw = (raw_df[c] for c in ['time_to_expiry', 'strike_value', 'iv'])

# 3D plot
fig = go.Figure(data=[
    go.Surface(z=sigma, x=T, y=K, opacity=0.75, name='interpolated'),
    go.Scatter3d(x=T_raw, y=K_raw, z=sigma_raw, mode='markers', marker=dict(size=2, color=np.log(T_raw)), name='raw')
    ])

fig.update_layout(
    title='implied volatility', 
    scene = dict(
        xaxis_title='x:expiry',
        yaxis_title='y:strike',
        zaxis_title='z:implied volatility'
        ),
    width=800, height=800, autosize=False,
    margin=dict(l=65, r=50, b=65, t=90))
fig.show()



# END